<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#library" data-toc-modified-id="library-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>library</a></span></li><li><span><a href="#data-load-및-classification-전처리" data-toc-modified-id="data-load-및-classification-전처리-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>data load 및 classification 전처리</a></span><ul class="toc-item"><li><span><a href="#data-load" data-toc-modified-id="data-load-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>data load</a></span></li><li><span><a href="#data-전처리" data-toc-modified-id="data-전처리-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>data 전처리</a></span></li></ul></li><li><span><a href="#머신러닝-K-fold-교차-검증(cross-validation)" data-toc-modified-id="머신러닝-K-fold-교차-검증(cross-validation)-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>머신러닝 K-fold 교차 검증(cross validation)</a></span><ul class="toc-item"><li><span><a href="#교차검증-결과-종합" data-toc-modified-id="교차검증-결과-종합-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>교차검증 결과 종합</a></span></li></ul></li></ul></div>

# library

In [2]:
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import os
import FinanceDataReader as fdr

import datetime as dt

import numpy as np

from tqdm import tqdm #진행바
from tqdm.notebook import tqdm

#시각화 라이브러리
import matplotlib as mpl
import matplotlib.pyplot as plt
print("matplotlib version: {}". format(mpl.__version__))

import seaborn as sns
print("seaborn version: {}". format(sns.__version__))


#한글설정
import matplotlib.font_manager as fm

font_dirs = ['/usr/share/fonts/truetype/nanum', ]
font_files = fm.findSystemFonts(fontpaths=font_dirs)

for font_file in font_files:
    fm.fontManager.addfont(font_file)
    
# 한글 출력을 위해서 폰트 옵션을 설정합니다.
# "axes.unicode_minus" : 마이너스가 깨질 것을 방지

sns.set(font="NanumBarunGothic", 
        rc={"axes.unicode_minus":False},
        style='darkgrid')


import warnings
warnings.filterwarnings('ignore')

# GPU
import tensorflow as tf

#https://www.tensorflow.org/guide/gpu#allowing_gpu_memory_growth
#프로세스의 요구량만큼 메모리 사용 설정
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if gpus:
    try:
        for i in range(len(gpus)):
            tf.config.experimental.set_memory_growth(gpus[i], True)
    except RuntimeError as e:
        # 프로그램 시작시에 메모리 증가가 설정되어야만 합니다
        print(e)
        
#지정해서 쓰고 싶으면 model 쓸 때는 with 구문 안에 넣기

matplotlib version: 3.4.2
seaborn version: 0.11.1
[]


# data load 및 classification 전처리

## data load

In [3]:
df = pd.read_csv('fin_df.csv')

In [4]:
df.head()

,code,date,D-9_open,D-9_high,D-9_low,D-9_close,D-9_volume,D-9_tradingvalue,D-9_change,D-9_ma_5,...,D-0_TSI,D-0_UO,D-0_SR,D-0_WR,D-0_AO,D-0_KAMA,D-0_ROC,D-0_PPO,D-0_PVO,next_change
0,10,19960920,1.003477,1.011599,1.000000,1.004618,17585,666446300,0.46,0.998593,...,9.148998,48.350147,31.747185,-68.252815,570.800000,0.989047,-3.823007,0.554780,4.094900,3.13
1,10,19961001,0.996490,1.010529,0.996490,1.010529,16636,622768400,1.05,1.003510,...,15.743477,54.854891,42.860061,-57.139939,1756.379412,0.951496,0.587415,1.295706,74.523403,0.23
2,10,19961007,0.996389,1.036139,0.981917,1.031315,24493,889436100,3.13,1.027704,...,3.767677,45.085356,44.637385,-55.362615,273.105882,1.030453,-0.347305,0.317907,27.495080,-2.33
3,10,19961129,0.947446,0.992123,0.947446,0.965846,10401,338695300,-3.42,0.961889,...,-39.769427,35.240071,25.167315,-74.832685,-3852.747059,1.045304,-9.317422,-4.774196,15.166891,-0.31
4,10,19961203,0.987512,1.006959,0.973625,0.979186,15542,484923300,-2.08,1.014173,...,-45.075455,36.539789,3.296207,-96.703793,-3866.444118,1.011865,-16.326848,-5.452434,15.938344,-3.90


In [5]:
df.shape

(1132308, 543)

## data 전처리

In [6]:
df['next_change'] = np.where(df['next_change']>0.05, 1, 0)

# 머신러닝 K-fold 교차 검증(cross validation)
데이터를 5-fold 교차검증으로 8개 모델을 학습하고 예측하여 평가한다.

In [9]:
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.dummy import DummyClassifier
import time

results=[]

def classification(model_):
    
    ## data load 및 전처리
    #df = pd.read_csv('fin_df.csv')
    #df['next_change'] = np.where(df['next_change']>0.05, 1, 0)
    
    # data split X, y
    X = df.iloc[:,2:-1] # code, date 뒤부터 nc 전까지
    y = df.iloc[:,-1] # nc
    i=0

    # Cross Validation
    sss = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
    for train_idx, test_idx in tqdm(sss.split(X,y)):
        x_train = df.iloc[train_idx,2:-1]
        y_train = df.iloc[train_idx,-1]
        x_test = df.iloc[test_idx,2:-1]
        y_test = df.iloc[test_idx,-1]
        
        _ = model_.fit(x_train, y_train)
        
        y_pred_train = model_.predict(x_train) 
        y_pred_test = model_.predict(x_test) # 예측
        accuracy_train = accuracy_score(y_train, y_pred_train)
        accuracy_test = accuracy_score(y_test, y_pred_test)
        roc_auc = roc_auc_score(y_test, y_pred_test)
        
        dummy = DummyClassifier(strategy = 'stratified', random_state = 1) # strategy 옵션: 'stratified', 'uniform', 'most_frequent'
        _ = dummy.fit(x_train, y_train)
        dummy_score = dummy.score(x_test, y_test)
        
        results.append([i,f'{str(model)[:str(model).index("(")]}',accuracy_train,accuracy_test, roc_auc, dummy_score])#, training_time
        i+=1
        
        

In [ ]:
%%time

# 1. Logistic regression    
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=1000)
temp = classification(model)


# 2. Decision tree
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
temp = classification(model)


# 3. Support vector machine
from sklearn.svm import SVC
model = SVC()
temp = classification(model)


# 4. Gaussian naive bayes
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
temp = classification(model)


# 5. K nearest neighbor
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier()
temp = classification(model)


# 6. Random forest
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
temp = classification(model)


# 7. Gradient boosing
from sklearn.ensemble import GradientBoostingClassifier
model = GradientBoostingClassifier()
temp = classification(model)


# 8. Neural network
from sklearn.neural_network import MLPClassifier
model = MLPClassifier(max_iter=1000)
temp = classification(model)


# results
# # 형태: i번째 학습, 학습 모델, train 정확도, test 정확도(최종정확도), roc_auc, dummy 학습에 걸린 시간

0it [00:00, ?it/s]

0it [00:00, ?it/s]

## 교차검증 결과 종합